In [30]:
import os
import time
import json
from PIL import Image, ImageDraw, ImageFont
import osmnx as ox
import matplotlib.pyplot as plt
from collections import defaultdict

# === Abbreviated city data ===
city_data = [
    {"nam": "Brasília, Brazil", "crd": (-15.793889, -47.882778), "dst": 2000, "grp": "Radial Power", "tax": "Centralized Burst"},
    {"nam": "Palmanova, Italy", "crd": (45.9061, 13.3095), "dst": 1000, "grp": "Radial Power", "tax": "Radial Convergence"},
    {"nam": "Versailles, France", "crd": (48.8049, 2.1204), "dst": 1600, "grp": "Radial Power", "tax": "Radial Implosion"},
    {"nam": "Karlsruhe, Germany", "crd": (49.0069, 8.4037), "dst": 1500, "grp": "Layered Hierarchies", "tax": "Centralized Ring"},
    {"nam": "Canberra, Australia", "crd": (-35.2811, 149.1287), "dst": 1800, "grp": "Layered Hierarchies", "tax": "Circled Globe"},
    {"nam": "Dubai, UAE", "crd": (25.056530, 55.207939), "dst": 1000, "grp": "Layered Hierarchies", "tax": "Segmented Radial Convergence"},
    {"nam": "Atlanta, USA", "crd": (33.891830, -84.259204), "dst": 800, "grp": "Linear Flows", "tax": "Flow Chart"},
    {"nam": "Manhattan, New York, USA", "crd": (40.7549, -73.9840), "dst": 1300, "grp": "Linear Flows", "tax": "Arc Diagram"},
    {"nam": "Chicago, USA", "crd": (41.8781, -87.6300), "dst": 1700, "grp": "Linear Flows", "tax": "Elliptical Implosion"},
    {"nam": "Greater London, UK", "crd": (51.5072, -0.1276), "dst": 2000, "grp": "Modular Networks", "tax": "Area Grouping"},
    {"nam": "Amsterdam, Netherlands", "crd": (52.3738, 4.8910), "dst": 1100, "grp": "Modular Networks", "tax": "Ramification"},
    {"nam": "Paris, France", "crd": (48.8566, 2.3522), "dst": 1500, "grp": "Modular Networks", "tax": "Scaling Circles"},
    {"nam": "Fez, Morocco", "crd": (34.065, -4.973), "dst": 1000, "grp": "Emergent Forms", "tax": "Organic Rhizome"},
    {"nam": "Old Delhi, India", "crd": (28.6562, 77.2410), "dst": 1200, "grp": "Emergent Forms", "tax": "Circular Ties"},
    {"nam": "Athens, Greece", "crd": (37.9755, 23.7348), "dst": 1300, "grp": "Emergent Forms", "tax": "Sphere"}
]

# === Directories ===
os.makedirs("images", exist_ok=True)
os.makedirs("comparisons", exist_ok=True)

# === Visual config ===
thumb_size = (600, 600)
font_size, title_font_size = 20, 26
panel_width = 2 * thumb_size[0] + 3 * 40
panel_height = thumb_size[1] + 3 * font_size + 80

try:
    font = ImageFont.truetype("arial.ttf", font_size)
    title_font = ImageFont.truetype("arial.ttf", title_font_size)
except:
    font = ImageFont.load_default()
    title_font = ImageFont.load_default()

# === Load list-based cache (with coords)
cache_path = "cache.json"
if os.path.exists(cache_path):
    with open(cache_path, "r") as f:
        cache = json.load(f)
else:
    cache = []

def is_cached(name, coords, dist):
    for entry in cache:
        if entry["nam"] == name and entry["dst"] == dist and tuple(entry["crd"]) == tuple(coords):
            return True
    return False

# === OSMnx settings ===
ox.settings.use_cache = True
ox.settings.overpass_endpoint = "https://overpass.kumi.systems/api"

# === Map generation ===
successful_maps = 0
updated_cache = []

for entry in city_data:
    name = entry["nam"]
    coords = entry["crd"]
    dist = entry["dst"]
    filename = name.replace(', ', '_').replace(' ', '_') + ".png"
    filepath = os.path.join("images", filename)

    if os.path.exists(filepath) and is_cached(name, coords, dist):
        print(f"🗂️  Map for {name} already exists – skipped.")
        updated_cache.append({"nam": name, "crd": coords, "dst": dist})
        continue

    try:
        print(f"🔄 Updating map for {name}...")
        G = ox.graph_from_point(coords, dist=dist, network_type="drive")
        G_proj = ox.project_graph(G)
        ox.plot_graph(
            G_proj,
            bgcolor="white",
            node_size=0,
            edge_color="black",
            edge_linewidth=0.3,
            show=False,
            save=True,
            filepath=filepath,
            dpi=300
        )
        plt.close("all")
        time.sleep(1)
        successful_maps += 1
        updated_cache.append({"nam": name, "crd": coords, "dst": dist})
    except Exception as e:
        print(f"⚠️  Failed to update {name}: {e}")

# === Save updated cache ===
with open(cache_path, "w") as f:
    json.dump(updated_cache, f, indent=2)

# === Group entries ===
grouped = defaultdict(list)
for entry in city_data:
    grouped[entry["grp"]].append(entry)

# === Generate PDF panels ===
comparison_images = []
successful_panels = 0

for group_name in sorted(grouped):
    title_img = Image.new("RGB", (panel_width, panel_height), "white")
    draw = ImageDraw.Draw(title_img)
    text_x = panel_width // 2 - draw.textlength(group_name, font=title_font) // 2
    draw.text((text_x, panel_height // 2), group_name, font=title_font, fill="black")
    comparison_images.append(title_img)

    for entry in grouped[group_name]:
        taxonomy = entry["tax"]
        name = entry["nam"]
        coords = entry["crd"]
        image_name = name.replace(', ', '_').replace(' ', '_') + ".png"
        tax_path = os.path.join("taxonomy", f"{taxonomy}.jpg")
        city_path = os.path.join("images", image_name)

        if not os.path.exists(tax_path) or not os.path.exists(city_path):
            print(f"❌ Missing file for {taxonomy} or {name}")
            continue

        tax_img = Image.open(tax_path).resize(thumb_size)
        city_img = Image.open(city_path).resize(thumb_size)

        panel = Image.new("RGB", (panel_width, panel_height), "white")
        draw = ImageDraw.Draw(panel)
        x1, x2, y = 40, thumb_size[0] + 2 * 40, 60
        panel.paste(tax_img, (x1, y))
        panel.paste(city_img, (x2, y))

        # Left: taxonomy
        draw.text((x1, y + thumb_size[1] + 10), taxonomy, font=font, fill="black")

        # Right: city name and coordinates
        draw.text((x2, y + thumb_size[1] + 10), name, font=font, fill="black")
        draw.text((x2, y + thumb_size[1] + 10 + font_size), f"({coords[0]:.4f}, {coords[1]:.4f})", font=font, fill="gray")

        comparison_images.append(panel)
        successful_panels += 1
        print(f"✅ Panel for {taxonomy} vs {name}")

# === Export PDF ===
if comparison_images:
    comparison_images[0].save(
        "comparison.pdf",
        save_all=True,
        append_images=comparison_images[1:], format="PDF"
    )
    print("\n📄 PDF exported to: comparison.pdf")
else:
    print("⚠️ No images to compile into PDF.")

# === Summary ===
print(f"\n✅ {successful_maps} maps updated or created.")
print(f"🖼️  {successful_panels} comparison panels created.")
print("🎯 Done.")

🗂️  Map for Brasília, Brazil already exists – skipped.
🗂️  Map for Palmanova, Italy already exists – skipped.
🗂️  Map for Versailles, France already exists – skipped.
🗂️  Map for Karlsruhe, Germany already exists – skipped.
🗂️  Map for Canberra, Australia already exists – skipped.
🗂️  Map for Dubai, UAE already exists – skipped.
🔄 Updating map for Atlanta, USA...
🗂️  Map for Manhattan, New York, USA already exists – skipped.
🗂️  Map for Chicago, USA already exists – skipped.
🗂️  Map for Greater London, UK already exists – skipped.
🗂️  Map for Amsterdam, Netherlands already exists – skipped.
🗂️  Map for Paris, France already exists – skipped.
🗂️  Map for Fez, Morocco already exists – skipped.
🗂️  Map for Old Delhi, India already exists – skipped.
🗂️  Map for Athens, Greece already exists – skipped.
✅ Panel for Organic Rhizome vs Fez, Morocco
✅ Panel for Circular Ties vs Old Delhi, India
✅ Panel for Sphere vs Athens, Greece
✅ Panel for Centralized Ring vs Karlsruhe, Germany
✅ Panel for C